<a href="https://colab.research.google.com/github/Tymass/tsp-solver/blob/main/TSP_genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSP - genetic based solver

Imports

In [1]:
import requests
import re
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from statistics import mean

Data download

In [2]:
url = 'https://www.math.uwaterloo.ca/tsp/vlsi/xqf131.tsp'
url2 = 'https://www.math.uwaterloo.ca/tsp/vlsi/xqg237.tsp'
url3 = 'https://www.math.uwaterloo.ca/tsp/vlsi/pma343.tsp'

def download_file(url):
    r = requests.get(url, allow_redirects=True)
    file_name = url.split("/")[-1]

    open(file_name, 'wb').write(r.content)

    return file_name

Data reading

In [ ]:
def read_tsplib95(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    coordinates = {}

    in_node_coord_section = False
    for line in lines:
        line = line.strip()

        if line.startswith("NODE_COORD_SECTION"):
            in_node_coord_section = True
            continue
        elif line == "EOF":
            break

        if in_node_coord_section:
            parts = re.split(r'\s+', line)
            node_id = int(parts[0])
            x_coord = float(parts[1])
            y_coord = float(parts[2])
            coordinates[node_id] = (x_coord, y_coord)

    return coordinates

file_name = download_file(url)
data = read_tsplib95(file_name)

Crossover algorithm

In [ ]:
def cycle_crossover(parent1, parent2):
    child1, child2 = parent1.copy(), parent2.copy()
    visited = [False] * len(parent1)
    cycle_num = 0

    while not all(visited):
        current_index = visited.index(False)
        cycle_start = parent1[current_index]
        while True:
            visited[current_index] = True
            cycle_num += 1
            next_city = parent2[current_index]
            next_index = parent1.index(next_city)

            if next_city == cycle_start:
                break

            current_index = next_index

        for i in range(len(parent1)):
            if visited[i] and cycle_num % 2 == 1:
                child1[i] = parent1[i]
                child2[i] = parent2[i]
            elif visited[i] and cycle_num % 2 == 0:
                child1[i] = parent2[i]
                child2[i] = parent1[i]

    return child1, child2

Mutation algorithms

In [ ]:
def scramble_mutation(route):
    start, end = sorted(random.sample(range(len(route)), 2))
    route_to_scramble = route[start:end]
    random.shuffle(route_to_scramble)
    return route[:start] + route_to_scramble + route[end:]

def inversion_mutation(route):
    start, end = sorted(random.sample(range(len(route)), 2))
    return route[:start] + route[start:end][::-1] + route[end:]

def insertion_mutation(route):
    route = route.copy()
    index = random.randrange(len(route))
    city = route.pop(index)
    route.insert(random.randrange(len(route)), city)
    return route

Main algorithm

In [ ]:
# Measure past distance
def calculate_total_distance(route, cities):
    total_distance = 0
    for i in range(len(route)):
        city_a = cities[route[i]]
        city_b = cities[route[i-1]]
        total_distance += np.linalg.norm(np.array(city_a) - np.array(city_b))
    return total_distance

# Assemble mutations
def mutate(route):
    mutation_type = random.choice([inversion_mutation, inversion_mutation, inversion_mutation])
    return mutation_type(route)

# Genetic algorithm
def genetic_algorithm(cities_dict, population_size, num_generations, mutation_rate):
    num_cities = len(cities_dict)
    population = [random.sample(list(cities_dict.keys()), num_cities) for _ in range(population_size)]

    selection_number = math.floor(population_size*0.1)
    elite_number = math.floor(population_size*0.05)
    resoults = []
    avg = []

    for generation in range(num_generations):
        print(f"Generation number: {generation}")
        population.sort(key=lambda route: calculate_total_distance(route, cities_dict))
        new_population = population[:elite_number]  # Elite

        if generation % 20 == 0:
            best_distance = calculate_total_distance(population[0], cities_dict)
            print(f"Best distance: {best_distance}")
            resoults.append((generation, best_distance))

            total_distance = sum(calculate_total_distance(route, cities_dict) for route in population)
            average_distance = total_distance / population_size
            avg.append((generation, average_distance))

        while len(new_population) < population_size:
            parent1, parent2 = random.choices(population[:selection_number], k=2)  # Selection
            child1, child2 = cycle_crossover(parent1, parent2)

            if random.random() < mutation_rate:
                child1 = mutate(child1)
                child2 = mutate(child2)

            new_population.extend([child1, child2])

        population = new_population

    return population[0], resoults, avg


# Parameters
population_size = 2000
num_generations = 700
mutation_rate = 0.3

best_route, learning_data, avg = genetic_algorithm(data, population_size, num_generations, mutation_rate)
best_route_distance = calculate_total_distance(best_route, data)

print("Best route:", best_route)
print("Total distance covered:", best_route_distance)

In [ ]:
#237

# Extract coords data
x_coords = [data[city][0] for city in data]
y_coords = [data[city][1] for city in data]

# Scatter for cities
plt.figure(figsize=(10, 6))
plt.scatter(x_coords, y_coords, color='blue')
plt.title('TSPLIB Points and Best Route Visualization')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)

# Plot the best route
for i in range(len(best_route)):
    start_city = data[best_route[i]]
    end_city = data[best_route[i - 1]]
    plt.plot([start_city[0], end_city[0]], [start_city[1], end_city[1]], color='red')

# Save image if you want
plt.savefig("route-100.png")
plt.show()

In [ ]:
# Plot route_lenght(iterations)

x = [data[0] for data in learning_data]
y = [data[1] for data in learning_data]

x_avg = [data[0] for data in avg]
y_avg = [data[1] for data in avg]

plt.plot(x,y)
plt.plot(x_avg,y_avg)
plt.xlabel('Iteration number')
plt.ylabel('Route length')
plt.grid("True")
plt.savefig("plot-100.png")
plt.show()

In [ ]:
# Save learning data for further analisys
with open('logs-100.txt', 'w') as file:
    for item in learning_data:
        file.write(str(item) + "\n")

    file.write("\n")

    for item in avg:
        file.write(str(item) + "\n")